<a href="https://colab.research.google.com/github/Tushar12S/Data-Science-Assignment-eCommerce-Transactions-Dataset/blob/main/Tushar_Shetty_Lookalike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler


In [8]:
# Load datasets
customers = pd.read_csv('/content/Customers.csv')
products = pd.read_csv('/content/Products.csv')
transactions = pd.read_csv('/content/Transactions.csv')

# Inspect column names to verify correctness
print(customers.columns)
print(products.columns)
print(transactions.columns)



Index(['CustomerID', 'CustomerName', 'Region', 'SignupDate'], dtype='object')
Index(['ProductID', 'ProductName', 'Category', 'Price'], dtype='object')
Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price'],
      dtype='object')


In [9]:
# Merge transactions with customers and products
merged_data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

# Rename ambiguous columns
merged_data.rename(columns={'Price_x': 'TransactionPrice', 'Price_y': 'ProductPrice'}, inplace=True, errors='ignore')

# Inspect merged data
print(merged_data.head())


  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  TransactionPrice     CustomerName         Region  SignupDate  \
0      300.68            300.68   Andrea Jenkins         Europe  2022-12-03   
1      300.68            300.68  Brittany Harvey           Asia  2024-09-04   
2      300.68            300.68  Kathryn Stevens         Europe  2024-04-04   
3      601.36            300.68  Travis Campbell  South America  2024-04-11   
4      902.04            300.68    Timothy Perez         Europe  2022-03-15   

                       ProductName     Category  ProductPrice  
0  ComfortLiving Bluetooth Speaker

In [10]:
# Aggregate transaction data
customer_features = merged_data.groupby('CustomerID').agg({
    'TotalValue': 'sum',  # Total revenue generated by the customer
    'Quantity': 'sum',    # Total quantity purchased
    'TransactionPrice': 'mean'  # Average price paid in transactions
}).reset_index()

# Merge with customer profile information
customer_profiles = customers.merge(customer_features, on='CustomerID', how='left')

# Fill missing values for customers with no transactions
customer_profiles.fillna(0, inplace=True)

# Display the resulting customer profiles
print(customer_profiles.head())


  CustomerID        CustomerName         Region  SignupDate  TotalValue  \
0      C0001    Lawrence Carroll  South America  2022-07-10     3354.52   
1      C0002      Elizabeth Lutz           Asia  2022-02-13     1862.74   
2      C0003      Michael Rivera  South America  2024-03-07     2725.38   
3      C0004  Kathleen Rodriguez  South America  2022-10-09     5354.88   
4      C0005         Laura Weber           Asia  2022-08-15     2034.24   

   Quantity  TransactionPrice  
0      12.0        278.334000  
1      10.0        208.920000  
2      14.0        195.707500  
3      23.0        240.636250  
4       7.0        291.603333  


In [11]:
# Normalize numerical columns
scaler = StandardScaler()
numerical_columns = ['TotalValue', 'Quantity', 'TransactionPrice']
customer_profiles[numerical_columns] = scaler.fit_transform(customer_profiles[numerical_columns])

# Display the normalized profiles
print(customer_profiles.head())


  CustomerID        CustomerName         Region  SignupDate  TotalValue  \
0      C0001    Lawrence Carroll  South America  2022-07-10   -0.051884   
1      C0002      Elizabeth Lutz           Asia  2022-02-13   -0.862714   
2      C0003      Michael Rivera  South America  2024-03-07   -0.393842   
3      C0004  Kathleen Rodriguez  South America  2022-10-09    1.035375   
4      C0005         Laura Weber           Asia  2022-08-15   -0.769499   

   Quantity  TransactionPrice  
0 -0.110735          0.110366  
1 -0.434049         -0.854626  
2  0.212579         -1.038306  
3  1.667493         -0.413708  
4 -0.919021          0.294836  


In [12]:
# Prepare feature matrix
feature_matrix = customer_profiles.set_index('CustomerID')[numerical_columns].values

# Compute cosine similarity
similarity_matrix = cosine_similarity(feature_matrix)

# Create a DataFrame
similarity_df = pd.DataFrame(similarity_matrix, index=customer_profiles['CustomerID'], columns=customer_profiles['CustomerID'])

# Display the similarity
print(similarity_df.head())


CustomerID     C0001     C0002     C0003     C0004     C0005     C0006  \
CustomerID                                                               
C0001       1.000000 -0.007041 -0.631946 -0.859581  0.856884  0.535879   
C0002      -0.007041  1.000000  0.778325 -0.488415  0.509339 -0.845597   
C0003      -0.631946  0.778325  1.000000  0.165898 -0.142181 -0.992820   
C0004      -0.859581 -0.488415  0.165898  1.000000 -0.989955 -0.052640   
C0005       0.856884  0.509339 -0.142181 -0.989955  1.000000  0.024587   

CustomerID     C0007     C0008     C0009     C0010  ...     C0191     C0192  \
CustomerID                                          ...                       
C0001       0.984218 -0.945042  0.823996 -0.382075  ...  0.931258  0.819770   
C0002      -0.114098 -0.243549  0.560490  0.926798  ...  0.339062  0.562188   
C0003      -0.699051  0.412339 -0.081705  0.956619  ... -0.316225 -0.075445   
C0004      -0.774871  0.963187 -0.985980 -0.128577  ... -0.956532 -0.976055   
C0005  

In [13]:
# Function to get top 3 lookalikes
def get_top_3_lookalikes(customer_id, similarity_df):
    similar_customers = similarity_df[customer_id].sort_values(ascending=False).iloc[1:4]
    return [(cust_id, score) for cust_id, score in similar_customers.items()]

# Generate recommendations for the first 20 customers
lookalikes = {}
for customer_id in customer_profiles['CustomerID'][:20]:
    lookalikes[customer_id] = get_top_3_lookalikes(customer_id, similarity_df)

# Display lookalike recommendations
print(lookalikes)


{'C0001': [('C0135', 0.9988906060990247), ('C0092', 0.9984804324524524), ('C0085', 0.995717591074867)], 'C0002': [('C0029', 0.9998501691627888), ('C0077', 0.9958951618516074), ('C0157', 0.9952987247900796)], 'C0003': [('C0111', 0.9987135659533447), ('C0190', 0.9958326787515313), ('C0038', 0.9894285881729176)], 'C0004': [('C0165', 0.9982215400873512), ('C0162', 0.9979217801509803), ('C0075', 0.9968180373404781)], 'C0005': [('C0167', 0.9999745104661253), ('C0020', 0.9997464179787114), ('C0128', 0.9987076575225469)], 'C0006': [('C0168', 0.9980204782883809), ('C0196', 0.9954421565049777), ('C0187', 0.9944493364789593)], 'C0007': [('C0125', 0.999915189711417), ('C0089', 0.9982699729142873), ('C0085', 0.9959368278047785)], 'C0008': [('C0084', 0.9961032994555747), ('C0113', 0.9959517012878036), ('C0017', 0.9932655818884806)], 'C0009': [('C0130', 0.9999592690544815), ('C0128', 0.9986532180739148), ('C0192', 0.9984645724439802)], 'C0010': [('C0176', 0.9994943208183413), ('C0055', 0.994145527790

In [14]:
# Prepare data for saving
lookalike_data = {'CustomerID': [], 'Lookalikes': []}
for cust_id, similar in lookalikes.items():
    lookalike_data['CustomerID'].append(cust_id)
    lookalike_data['Lookalikes'].append(similar)

# Create a DataFrame for lookalikes
lookalike_df = pd.DataFrame(lookalike_data)

# Save as CSV
lookalike_df.to_csv('Lookalike.csv', index=False)

# Display the saved DataFrame
print(lookalike_df.head())


  CustomerID                                         Lookalikes
0      C0001  [(C0135, 0.9988906060990247), (C0092, 0.998480...
1      C0002  [(C0029, 0.9998501691627888), (C0077, 0.995895...
2      C0003  [(C0111, 0.9987135659533447), (C0190, 0.995832...
3      C0004  [(C0165, 0.9982215400873512), (C0162, 0.997921...
4      C0005  [(C0167, 0.9999745104661253), (C0020, 0.999746...
